In [1]:
import numpy as np, matplotlib.pyplot as plt
from scipy.integrate import simpson
import pandas as pd, seaborn as sb

from anianssonwall.micellethermo import *
from anianssonwall.anianssonwall import *

In [2]:
def dimer_energy(eq, n1, n2):
    assert np.all(n1 >= 1)
    assert np.all(n2 >= 1)
    G1, G2 = eq.free_energy(n1), eq.free_energy(n2)
    return G1 + G2

def dimer_energy2(eq, n1, n2):
    assert np.all(n1 >= 1)
    assert np.all(n2 >= 1)

    N = n1 + n2
    na = np.floor(n1)
    nb = N - 1 - na
    nb[n1 > n2] = np.floor(n2[n1 > n2])
    na[n1 > n2] = N[n1 > n2] - 1 - nb[n1 > n2]

    assert np.all(np.isclose(na + nb, N-1))
    G_minimum = eq.free_energy(n1) + eq.free_energy(n2)
    G_barrier = eq.free_energy(na) + eq.free_energy(nb)

    # analytically continue to barriers to show as a continuum
    x = n1 - np.floor(n1)
    return G_minimum + (4*x*(1-x))**4 * (G_barrier - G_minimum)

In [ ]:
ratio = 0.2
free = 0.01
n_mode = 20,
density = 0.01
g0 = 0.5

eq = MicelleDistribution.from_fit(ratio, free, n_mode, density, g0=g0)
model = AnianssonWallMaibaum(distribution=eq)

n = model.n
ceq = model.steady_state(n)
phi = np.sum(n * ceq)
navg = phi / np.sum(ceq)
assert (phi - density)**2 < 1e-8

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(3.375, 3.375),
                                height_ratios=[1, 1])

n1, n2 = round(eq.local_maximum), round(eq.local_minimum)
G1, G2 = [eq.free_energy(nn) for nn in [n1, n2]]
pl, = ax2.step(n, eq.free_energy(n), where='mid', label=r'$\mu=0$')
pl, = ax1.step(n, eq.free_energy(n), where='mid', label=r'single')
for ax in [ax1, ax2]:
    ax.plot([n1, n2], [G1, G2], 'o', c=pl.get_color(), mfc='None')

ax1.plot(np.nan, np.nan, '', c='None', label='double')

first = True
for N in np.flipud(np.arange(15, 31, 5)):
    label = f'{N}'
    if first:
        label = f'$N={label}$'
        first = False

    n_dimer = np.arange(1,N)
    dG = dimer_energy(eq, n_dimer, N - n_dimer)/2
    ax1.step(n_dimer, dG, where='mid', label=label)

for mu in np.linspace(5e-2, 1.5e-1, 3):
    label = f'{mu:.2f}'
    ax2.step(n, np.exp(mu*n) * eq.free_energy(n), where='mid', label=label)

for ax in [ax1, ax2]:
    ax.set_xlim([0, 40])
    ax.set_xlabel('$n$')

ax1.legend(loc='lower right')
ax2.legend(loc='best')
ax1.set_ylim([0, 2.5])
ax2.set_ylim([0, 10])
ax1.set_ylabel(r'$\langle \Delta G \rangle$')
ax2.set_ylabel(r'$\Delta G$')

plt.show()


In [ ]:
ratio = 0.2
free = 0.01
n_mode = 20
density = 0.01
g0 = 0.5

eq = MicelleDistribution.from_fit(ratio, free, n_mode, density, g0=g0)
model = AnianssonWallMaibaum(distribution=eq)

n = model.n
n = np.linspace(1, n[-1], 1000)

plt.figure(figsize=(3.375, 3))
ax1 = plt.gca()

n1, n2 = eq.local_maximum, eq.local_minimum
G1, G2 = [eq.free_energy(nn) for nn in [n1, n2]]
pl, = ax1.plot(n, eq.free_energy(n), '--', label=r'single')
ax1.plot([n1, n2], [G1, G2], 'o', c=pl.get_color(), mfc='None')

ax2 = plt.twinx()

ax2.plot(np.nan, np.nan)

first = True
for N in [30]:
    label = f'{N}'
    if first:
        label = f'$N={label}$'
        first = False

    # n_dimer = np.linspace(1, N-1, 2*(N-1)-1)
    n_dimer = np.linspace(1, N-1, 10000)
    dG = dimer_energy(eq, n_dimer, N - n_dimer)
    pl, = ax2.plot(n_dimer, dG, '--', label=label)
    dG = dimer_energy2(eq, n_dimer, N - n_dimer)
    ax2.plot(n_dimer, dG, c=pl.get_color())

ax2.set_ylabel(r'$\Delta G_2$', c=pl.get_color())
ax2.tick_params(axis='y', labelcolor=pl.get_color())

ax1.set_xlim([0, 40])
ax1.set_xlabel('$n$')

ax1.set_ylim([0, 2.5])
ax2.set_ylim([1, 3.5])
ax1.set_ylabel(r'$\Delta G_1$')
ax2.set_ylabel(r'$\Delta G_2$')

plt.show()

This example shows how timescale separation vanishes as we increase the size of aggregates exchanged:

In [ ]:
ratio = 0.2
free = 0.01
n_mode = 20,
density = 0.01
g0 = 0.5

eq = MicelleDistribution.from_fit(ratio, free, n_mode, density, g0=g0)

# fig, axes = plt.subplots(nrows=10, figsize=(3.375, 4.5), sharex=True)
plt.figure(figsize=(3.375, 2*3.375))
ax1 = plt.gca()

evals = dict()

for m in range(1, 16):
    model = AnianssonWallMaibaum(distribution=eq, max_n_exchanged=m)
    ceq = model.steady_state(model.n)
    evals[m] = np.log10(-model.characteristic_rates(ceq))

sb.boxplot(ax=ax1, data=evals, showfliers=False, orient='h',
            width=0.8, linewidth=0.5, boxprops={'facecolor':'None'})
sb.swarmplot(ax=ax1, data=evals, orient='h',
            edgecolor=['k']*len(evals), linewidth=0.25, size=2.5)

import matplotlib.ticker as ticker
ax1.xaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: rf'$10^{{{y:g}}}$'))
ax1.spines[['right', 'top']].set_visible(False)
# ax1.set_yticks([])
ax1.set_xlabel(r'$|\lambda_i|$')
ax1.set_ylabel('max $n$ exchanged')
ax1.set_xlim([-6.25, -1])

plt.figure()
ax2 = plt.gca()
ax2.step(model.n, ceq, where='mid')
ax2.set_xlabel('$n$')
ax2.set_ylabel('$c_\mathrm{eq}$')

plt.show()

In [ ]:
ratio = 0.2
free = 0.01
density = 0.01
g0 = 0.5

fig = plt.figure(figsize=(3.375, 3.375))
ax1 = plt.gca()
from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax1)
ax2 = divider.append_axes('bottom', size='100%', pad=0)
ax3 = divider.append_axes('top', size='100%', pad=0.45)

e0 = []
e1 = []
e2 = []
e_theory1 = []
e_theory2 = []
e_theory3 = []
n_mode_attempt = [20, 30, 40, 50, 60, 70, 80, 90, 100, 110]
n_mode_list = []

for n_mode in n_mode_attempt:
    eq = MicelleDistribution.from_fit(ratio, free, n_mode, density, g0=g0)
    model = AnianssonWallMaibaum(distribution=eq)
    g0 = eq.g
    n_mode_list += [eq.local_minimum]

    n = model.n
    ceq = model.steady_state(n)

    C = np.sum(ceq)
    phi = np.sum(n * ceq)
    assert (phi - density)**2 < 1e-8
    p = ceq / C

    # Exact final decay mode as slowest eigenvector in steady-state:
    J = model.jacobian_rhs(ceq)
    evals, V = np.linalg.eig(J)
    # print(evals)
    order = np.argsort(np.abs(evals))
    evals, V = evals[order], V[:,order]
    # print(evals)
    assert np.abs(evals[0]) < 1e-6 # steady-state mode


    kf, kb = model.rate_coeffs
    nsaddle = np.round(eq.local_maximum)
    index_saddle = np.where(n == nsaddle)[0][0]
    nsaddle = n[index_saddle]
    nmode = np.round(eq.local_minimum)
    index_mode = np.where(n == nmode)[0][0]
    nmode = n[index_mode]
    kf_saddle = kf[index_saddle]
    kb_saddle = kb[index_saddle]
    c_saddle = ceq[index_saddle]
    # k = kb_saddle / kf_saddle
    # k = p[0] * kb_saddle
    # k = kb_saddle
    print(f'{kf_saddle:<8.5g} {kb_saddle:<8.5g} {c_saddle:<8.5g}')
    # print(np.array((kf, kb)).T)
    # k = kf[0]
    e0 += [evals[1]]
    e1 += [evals[2]]
    e2 += [evals[3]]
    F1 = eq.free_energy(nsaddle)

    select = n < nsaddle
    pfree = p[select] / np.sum(p[select])
    Sfree = -np.sum(pfree*np.log(pfree))
    # F0 = eq.free_energy(1) - Sfree
    U0 = np.sum(pfree * eq.free_energy(n[select]))
    F0 = U0 - Sfree
    # k = np.sum(p[n < nsaddle]) * kf_saddle * phi**2
    k = kf_saddle * phi**2
    k = 1
    e_theory1 += [k * np.exp(-(F1 - F0))]

    select = n > nsaddle
    pagg = p[select] / np.sum(p[select])
    Sagg = -np.sum(pagg*np.log(pagg))
    # navg_agg = np.sum(n[select] * p[select]) / np.sum(p[select])
    # navg2_agg = np.sum(n[select]**2 * p[select]) / np.sum(p[select])
    # variance = navg2_agg - navg_agg**2
    # S = 0.5 * np.log(2*np.pi*np.exp(1)*variance)
    U0 = np.sum(pagg * eq.free_energy(n[select]))
    F0 = U0 - Sagg
    # k = np.sum(p[n > nsaddle]) * kb_saddle
    k = kb_saddle
    k = 1
    e_theory2 += [k * np.exp(-(F1 - F0))]
    F0 = eq.free_energy(nmode)
    F0 -= 0.5*np.log((nmode - nsaddle)/(F1-F0))
    e_theory3 += [k * np.exp(-(F1 - F0))]
    # evals = evals[1:]
    # evals = np.log10(-evals)

    V[:,0] *= np.sign(V[0,0])
    V[:,0] *= np.sum(n*ceq) / np.sum(V[:,0])
    V[:,1] *= np.sign(V[0,1])
    V[:,1] *= C / np.linalg.norm(V[:,1])
    assert np.allclose(V[:,0], n*ceq, rtol=1e-8, atol=1)
    assert np.isclose(np.sum(n*V[:,1]), 0, rtol=1e-8, atol=1)

    pl, = ax1.step(n, ceq, where='mid')
    # ax1.plot(eq.local_maximum, model.steady_state(eq.local_maximum),
    #          'o', mfc='w', c=pl.get_color())
    # ax1.plot(eq.local_minimum, model.steady_state(eq.local_minimum),
    #          'o', c=pl.get_color())

    navg = np.sum(p * n)
    v = ceq * (n - navg)
    tangent = n*ceq / np.sum(n**2 * ceq)
    v -= n.dot(v) * tangent
    # v *= C / simpson(v**2, n)**0.5
    assert np.isclose(n.dot(tangent), 1.)
    assert np.isclose(n.dot(v), 0.)

    # n = np.linspace(1, np.max(n), 1000)
    # c = model.steady_state(n)
    # p = c / simpson(c, n)

    # navg = simpson(p * n, n)
    # v = c * (n - navg)
    # tangent = n*c / simpson(n**2 * c, n)
    # v -= simpson(n*v, n) * tangent
    # # v *= C / simpson(v**2, n)**0.5
    # assert np.isclose(simpson(n * tangent, n), 1.)
    # assert np.isclose(simpson(n * v, n), 0.)

    v *= np.max(V[10:,1]) / np.max(v)
    # v *= C / np.linalg.norm(v)

    pl, = ax2.step(model.n, V[:,1], where='mid')
    ax2.plot(n, v, '--', c=pl.get_color())


# plt.figure(figsize=(3.375, 2.1))
# ax3 = plt.gca()
n_mode_list = np.array(n_mode_list)
e0 = np.array(e0)
e1 = np.array(e1)
e2 = np.array(e2)
e_theory1 = -np.array(e_theory1)
e_theory2 = -np.array(e_theory2)
e_theory3 = -np.array(e_theory3)
e_theory1 *= e0[-2] / e_theory1[-2]
e_theory2 *= e0[-2] / e_theory2[-2]
e_theory3 *= e0[-2] / e_theory3[-2]
# print(e0)
# print(e1)
# print(e2)
print(np.array((n_mode_list, e_theory1 / e0, e_theory2 / e0)).T)
ax3.semilogy(n_mode_list[:-1], -e0[:-1], label='exact')
# ax3.semilogy(n_mode_list, -e1)
# ax3.semilogy(n_mode_list, -e2)
ax3.semilogy(n_mode_list, -e_theory3, '-.',
                label=r'$1/\tau_2$ using Eq.\ (7)')
ax3.semilogy(n_mode_list, -e_theory2, '--',
                label=r'$\exp{(-\beta(W_{n^*} - F_\text{agg}))}$')
ax3.semilogy(n_mode_list, -e_theory1, ':',
                label=r'$\exp{(-\beta(W_{n^*} - F_\text{free}))}$')
ax3.set_xlabel(r'modal aggregate number $\bar{n}$')
ax3.set_ylabel(r'$|\lambda_\text{slow}^\infty| \sim 1/\tau_2$')
ax3.set_xlim([0, 110])
ax3.legend(loc='best')

ax1.set_ylabel(r'$c_n^\mathrm{eq}$')
ax2.set_ylabel(r'$v_n^\infty$')
ax1.set_ylim([0, 3.5e-5])
# ax2.set_ylim([-0.35, 0.2])
ax2.set_ylim([-2e-4, 1.25e-4])

ax1.set_xticklabels([])
for ax in [ax1, ax2]: ax.set_xlim([0, 130])
ax2.set_xlabel('$n$')

for ax, letter in zip([ax3, ax1, ax2], 'abc'):
    label = ax.text(0.95, 0.725, rf'\textbf{{{letter}}}',
                    transform=ax.transAxes,
                    ha='center', va='bottom', fontsize=18)
    label.set_in_layout(False)

plt.show()